In [ ]:
import os
import math
import random
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
import time
import multiprocessing
from concurrent.futures import ProcessPoolExecutor

# Set precision
getcontext().prec = 30
scale_factor = Decimal("1")

# Base coordinates for numpy optimization
trunk_w = 0.15
trunk_h = 0.2
base_w = 0.7
mid_w  = 0.4
top_w  = 0.25
tip_y = 0.8
tier_1_y = 0.5
tier_2_y = 0.25
base_y = 0.0
trunk_bottom_y = -trunk_h

BASE_COORDS = np.array([
    (0.0, tip_y),
    (top_w / 2, tier_1_y),
    (top_w / 4, tier_1_y),
    (mid_w / 2, tier_2_y),
    (mid_w / 4, tier_2_y),
    (base_w / 2, base_y),
    (trunk_w / 2, base_y),
    (trunk_w / 2, trunk_bottom_y),
    (-(trunk_w / 2), trunk_bottom_y),
    (-(trunk_w / 2), base_y),
    (-(base_w / 2), base_y),
    (-(mid_w / 4), tier_2_y),
    (-(mid_w / 2), tier_2_y),
    (-(top_w / 4), tier_1_y),
    (-(top_w / 2), tier_1_y),
])

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        self._polygon = None

    @property
    def polygon(self):
        if self._polygon is None:
            rad = np.radians(self.angle)
            c, s = np.cos(rad), np.sin(rad)
            R = np.array([[c, -s], [s, c]])
            new_coords = BASE_COORDS @ R.T + np.array([self.center_x, self.center_y])
            self._polygon = Polygon(new_coords)
        return self._polygon
    
    def update(self, x, y, angle):
        self.center_x = x
        self.center_y = y
        self.angle = angle
        self._polygon = None # Invalidate cache

    def clone(self):
        return ChristmasTree(self.center_x, self.center_y, self.angle)

def parse_csv(csv_path):
    df = pd.read_csv(csv_path)
    df["x"] = df["x"].astype(str).str.strip().str.lstrip("s")
    df["y"] = df["y"].astype(str).str.strip().str.lstrip("s")
    df["deg"] = df["deg"].astype(str).str.strip().str.lstrip("s")
    df[["group_id", "item_id"]] = df["id"].str.split("_", n=2, expand=True)
    
    dict_of_tree_list = {}
    for group_id, group_data in df.groupby("group_id"):
        tree_list = [
            ChristmasTree(center_x=row["x"], center_y=row["y"], angle=row["deg"])
            for _, row in group_data.iterrows()
        ]
        dict_of_tree_list[int(group_id)] = tree_list
    return dict_of_tree_list

def write_submission(dict_of_tree_list, out_file):
    rows = []
    sorted_keys = sorted(dict_of_tree_list.keys())
    for group_name in sorted_keys:
        tree_list = dict_of_tree_list[group_name]
        for item_id, tree in enumerate(tree_list):
            rows.append(
                {
                    "id": f"{group_name}_{item_id}",
                    "x": f"s{tree.center_x}",
                    "y": f"s{tree.center_y}",
                    "deg": f"s{tree.angle}",
                }
            )
    pd.DataFrame(rows).to_csv(out_file, index=False)

def get_bbox_side(trees):
    min_x, min_y = float('inf'), float('inf')
    max_x, max_y = float('-inf'), float('-inf')
    
    for t in trees:
        bounds = t.polygon.bounds
        min_x = min(min_x, bounds[0])
        min_y = min(min_y, bounds[1])
        max_x = max(max_x, bounds[2])
        max_y = max(max_y, bounds[3])
        
    return max(max_x - min_x, max_y - min_y)

def has_overlap(trees):
    polys = [t.polygon for t in trees]
    tree = STRtree(polys)
    # This is still the bottleneck. 
    # Optimization: Only check the moved tree against others?
    # But for full validity check we need all.
    # For SA move, we can optimize.
    
    for i, poly in enumerate(polys):
        indices = tree.query(poly)
        for idx in indices:
            if idx != i:
                if poly.intersection(polys[idx]).area > 1e-9:
                    return True
    return False

def optimize_n(args):
    n, trees, n_iter, initial_temp, cooling_rate = args
    
    # Re-seed random for multiprocessing
    np.random.seed(int(time.time() * 1000) % 2**32 + n)
    random.seed(int(time.time() * 1000) % 2**32 + n)
    
    current_trees = [t.clone() for t in trees]
    current_side = get_bbox_side(current_trees)
    best_trees = [t.clone() for t in current_trees]
    best_side = current_side
    
    temp = initial_temp
    
    # Pre-build STRtree for static trees? No, all can move.
    
    for i in range(n_iter):
        # Pick a tree
        idx = random.randint(0, len(current_trees) - 1)
        t = current_trees[idx]
        
        # Backup state
        old_x, old_y, old_angle = t.center_x, t.center_y, t.angle
        
        # Perturb
        shift_scale = temp * 0.5
        dx = random.uniform(-shift_scale, shift_scale)
        dy = random.uniform(-shift_scale, shift_scale)
        d_angle = random.uniform(-10 * temp, 10 * temp)
        
        # Apply move
        t.update(old_x + dx, old_y + dy, old_angle + d_angle)
        
        # Check overlap ONLY for this tree
        # We need to check if t overlaps with any other tree in current_trees
        # Construct list of OTHER polygons
        others = [ot.polygon for j, ot in enumerate(current_trees) if j != idx]
        
        overlap = False
        # Simple check first: bounding box intersection
        t_poly = t.polygon
        
        # Use STRtree for others if N is large?
        # For N < 50, linear scan might be faster than building tree every time
        # But we can keep a persistent STRtree and update it? shapely STRtree is immutable.
        
        # Linear scan with bounds check first
        t_bounds = t_poly.bounds
        for other_poly in others:
            o_bounds = other_poly.bounds
            if (t_bounds[0] > o_bounds[2] or t_bounds[2] < o_bounds[0] or
                t_bounds[1] > o_bounds[3] or t_bounds[3] < o_bounds[1]):
                continue
            if t_poly.intersection(other_poly).area > 1e-9:
                overlap = True
                break
        
        if overlap:
            # Revert
            t.update(old_x, old_y, old_angle)
            continue
            
        # Valid move, check score
        candidate_side = get_bbox_side(current_trees)
        
        delta = candidate_side - current_side
        if delta < 0 or random.random() < math.exp(-delta / temp):
            current_side = candidate_side
            if current_side < best_side:
                best_side = current_side
                best_trees = [ct.clone() for ct in current_trees]
                # print(f"N={n} Iter {i}: {best_side}")
        else:
            # Revert
            t.update(old_x, old_y, old_angle)
            
        temp *= cooling_rate
        
    return n, best_trees, best_side

def main():
    print("Loading submission...")
    tree_dict = parse_csv("submission.csv")
    
    # Target Ns: Focus on small Ns first where efficiency is low
    # And run in parallel
    target_ns = list(range(1, 21)) # 1 to 20
    
    tasks = []
    for n in target_ns:
        if n in tree_dict:
            # More iterations for smaller N
            n_iter = 20000 if n < 10 else 5000
            tasks.append((n, tree_dict[n], n_iter, 0.5, 0.995))
            
    print(f"Starting optimization for {len(tasks)} tasks with {multiprocessing.cpu_count()} cores...")
    
    improved_dict = tree_dict.copy()
    
    with ProcessPoolExecutor() as executor:
        results = executor.map(optimize_n, tasks)
        
        for n, best_trees, best_side in results:
            print(f"N={n} Final Side: {best_side}")
            improved_dict[n] = best_trees
            
    print("Writing submission...")
    write_submission(improved_dict, "submission_sa_parallel.csv")

if __name__ == "__main__":
    main()
